# Exploring Commonsense Reasoning in Language Models: Walk vs Drive for Car Washing

Large language models (LLMs) are increasingly used to provide everyday recommendations. However, it remains unclear whether these models consistently apply real-world commonsense when making practical decisions, especially when contextual task requirements conflict with simple surface heuristics.

In this experiment, I evaluate whether an LLM demonstrates commonsense reasoning in a simple but intentionally tricky scenario. I ask the model whether a person should walk or drive to a car wash located 50 meters from their home. While short distances usually favor walking, washing a car implicitly requires bringing the vehicle to the destination, making driving the more realistic and task-consistent choice.

I test five prompt variations that describe the same scenario but use different framings and constraints. The goal of this experiment is to:
- Evaluate whether the model applies task-relevant commonsense
- Examine how prompt framing influences recommendations
- Identify cases where the model produces inconsistent or misleading decisions

All experiments are conducted using an LLM (kimi-k2.5, ranked 1st in OpenRouter) accessed through the OpenRouter API, and all results are displayed directly within this notebook.


In [ ]:
import requests
import json
import time

# -------- API Key --------
OPENROUTER_API_KEY = "Enter OpenRouter API Key"

MODEL_NAME = "moonshotai/kimi-k2.5"
TEMPERATURE = 0.7
DISTANCE = 50


# -------- Prompt Set --------
PROMPTS = {
    "Neutral": f"""
I need to go to a car wash. The car wash is {DISTANCE} meters away from my home.
Would you recommend walking or driving? Explain briefly.
""",

    "Convenience": f"""
My goal is convenience and saving time.
The car wash is {DISTANCE} meters away from my home.
Should I walk or drive? Explain.
""",

    "Health": f"""
I want to make a healthy and environmentally friendly choice.
The car wash is {DISTANCE} meters away from my home.
Should I walk or drive? Explain.
""",

    "Binary": f"""
Distance to car wash: {DISTANCE} meters.
Answer only WALK or DRIVE.
""",

    "Quantitative": f"""
If a destination is {DISTANCE} meters away, should a person walk or drive?
Assume normal weather and average walking speed. Explain your reasoning.
"""
}


# -------- Query Function --------
def query_model(prompt):

    url = "https://openrouter.ai/api/v1/chat/completions"

    headers = {
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": MODEL_NAME,
        "messages": [{"role": "user", "content": prompt}],
        "temperature": TEMPERATURE
    }

    for _ in range(3):
        try:
            response = requests.post(url, headers=headers, data=json.dumps(payload))
            return response.json()["choices"][0]["message"]["content"]
        except:
            time.sleep(2)

    return "ERROR"


# -------- Run All Prompts --------
for name, prompt in PROMPTS.items():

    print("\nPROMPT:", name)

    output = query_model(prompt)

    print(output)
    print()

PROMPT: Neutral

 If you need to **wash your car**, you must **drive** it there—though such a short trip (50 meters is roughly a 1-minute walk) means your engine won't fully warm up, which isn't ideal for the vehicle.

If you're going for any other reason (e.g., to pay, ask about services, or buy supplies), **walk**. It's only about a minute away, making driving more hassle than it's worth with parking and startup time.



PROMPT: Convenience

 **Walk.**

At 50 meters (about a 30–45 second walk), driving will actually cost you more time and create less convenience than walking.

**Time breakdown:**
- **Walking**: ~1 minute round trip (no preparation needed)
- **Driving**: ~3–5 minutes total when you factor in getting in/out, starting the engine, buckling up, navigating the lot, and parking

**Convenience factors:**
- No need to find a parking spot or maneuver in a tight car wash lot
- No engine warm-up issues (short trips like this are actually bad for your car's engine and battery)
- Zero fuel cost or emissions for such a negligible distance

**The obvious caveat:** If you're taking your *car* to be washed, you obviously need to drive it there. However, since it's only 50 meters away, the most time-efficient approach is to **drive the car there, drop it off, and walk back home** (or wait outside and walk home afterward) rather than sitting in the car idling or circling the block. If it's a self-service wash, just drive it over—accepting that the "trip" portion takes seconds, but the car prep/parking still makes it slower than if you could teleport.

**Bottom line:** For a destination you can see from your driveway, walking wins on both speed and convenience—unless you need to transport a vehicle there.



PROMPT: Health

 **Walk.**

For a distance of only **50 meters** (about a 1-minute walk), driving is environmentally wasteful and offers zero health benefits, while walking achieves both your goals perfectly.

**Environmental reasons:**
- **Cold-start emissions**: The first few minutes of driving produce the highest emissions because the engine and catalytic converter aren't at optimal temperature. Driving 50 meters means nearly 100% of your trip is high-pollution cold-start phase.
- **Total footprint**: Starting the engine, idling, and parking likely take longer than the drive itself, creating emissions for essentially zero distance traveled.

- **Sedentary vs. active**: Driving 50 meters contributes to sedentary time; walking provides light physical activity, however brief.
- **Air quality**: You'll avoid breathing the concentrated emissions from your own car's exhaust during startup.

**Practical note:**
If you're going to the car wash **to wash your car**, you obviously need to drive the vehicle there. However, given it's only 50 meters away, consider:
- Walking over first to pay/check availability, then driving the car over only when ready
- Combining the car wash with a longer trip rather than making a special 100-meter round trip just to wash it

**Bottom line:** For your personal transportation to any destination 50 meters away, walking is the unequivocally superior choice for both health and the environment.



PROMPT: Binary

WALK



PROMPT: Quantitative

For a destination only **50 meters away**, a person should **walk**.

Here is the reasoning:

**Time Efficiency**
- **Walking**: At an average pace of 5 km/h (3.1 mph), walking 50 meters takes approximately **35–40 seconds**.
- **Driving**: While the drive itself might take only 10–15 seconds at low speed, the total time includes walking to the car, starting the engine, buckling up, finding parking (which will likely be farther than 50 meters away), and walking from the car to the destination. This easily totals **3–5 minutes or more**, making driving significantly slower for such a short distance.

**Practicality**
- 50 meters is roughly half the length of a soccer field or about 2–3 city bus lengths. In many cases, the nearest legal parking spot will be farther from your destination than the starting point is.
- The effort required to get into a vehicle, navigate traffic, and park outweighs the minimal effort of a 35-second stroll.

**Health & Environmental Impact**
- Walking provides brief cardiovascular activity and muscle engagement with zero emissions.
- Driving produces unnecessary fuel consumption and pollution for a distance most people can cover in less than a minute.

The only reasons to drive 50 meters would involve significant mobility limitations, transporting heavy/bulky cargo that cannot be carried, or hazardous conditions not covered by "normal weather" (e.g., carrying an injured person). For an able-bodied person under normal circumstances, driving this distance is impractical and inefficient.

## Failure Cases

The outputs for each prompt are shown directly in the notebook results above. While the scenario remained identical across prompts, the model demonstrated several patterns that conflict with expected real-world commonsense.

### Failure Case 1: Overreliance on Distance-Based Heuristics

The most prominent failure is the model’s tendency to rely on a simple distance heuristic. Across multiple prompts, the model concluded that walking is preferable because the destination is only 50 meters away. However, visiting a car wash typically requires bringing the vehicle to the location. From a real-world perspective, driving is therefore the more contextually appropriate action.

For example, in the Quantitative prompt, the model provided detailed time and efficiency calculations supporting walking. While the reasoning is logically consistent within a general transportation context, it ignores the implicit task requirement of washing a car. This suggests that the model prioritizes generic commonsense patterns over task-specific reasoning.

### Failure Case 2: Conflicting Interpretations of the Task

In some responses, particularly the Neutral prompt, the model simultaneously acknowledged that driving is required to wash the car while still presenting walking as a valid recommendation depending on interpretation. This indicates that the model attempts to generate multiple plausible interpretations rather than committing to the most realistic scenario. Such ambiguity may be useful conversationally but can reduce decision reliability.

### Failure Case 3: Artificial Confidence Under Output Constraints

The Binary prompt forced the model to provide a single-word response. Under this constraint, the model returned a confident answer (“WALK”) without acknowledging contextual ambiguity. This suggests that restricting output format can remove uncertainty from the response without improving reasoning accuracy.

## Reflection

This experiment suggests that the model relies heavily on dominant commonsense heuristics rather than consistently applying contextual task reasoning. The model appears capable of recognizing relevant real-world constraints but does not always prioritize them when generating final recommendations.

The results also demonstrate that prompt framing significantly influences model reasoning. Prompts emphasizing efficiency, health, or quantitative analysis consistently reinforced the distance-based heuristic, while more open-ended prompts allowed the model to partially recognize task-specific context.

Another observation is that the model often generates explanations that appear logically detailed but may not fully align with real-world practical reasoning. This behavior suggests that the model optimizes for producing plausible and coherent explanations rather than performing structured decision analysis.